# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

scaler = StandardScaler()

# Scale X_train and X_val using StandardScaler

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)

# Your code here

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both X_train and X_val
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Convert the scaled arrays back to DataFrames, preserving the original column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)


In [4]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data

# Calculate a baseline r-squared score on training data

# Your code here

# Create a LinearRegression model and fit it on scaled training data
linregr = LinearRegression()
linregr.fit(X_train_scaled, y_train)
# Calculate a baseline r-squared score on training data
baseline_r2 = linregr.score(X_train_scaled, y_train)

# Print the baseline R-squared score
print("Baseline R-squared score:", baseline_r2)

Baseline R-squared score: 0.7868344817421309


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [5]:
# Your code here

# Set up data structure


# Find combinations of columns and loop over them

    # Make copies of X_train and X_val
    
    
    # Add interaction term to data

    
    # Find r-squared score (fit on training data, evaluate on validation data)

    
    # Append to data structure
    
    
# Sort and subset the data structure to find the top 7

from itertools import combinations
import pandas as pd

# Make copies of X_train and X_val
X_train_interactions = X_train_scaled.copy()
X_val_interactions = X_val_scaled.copy()

# Data structure to store results
interaction_scores = []

# Loop over all possible pairs of columns in X_train
for col1, col2 in combinations(X_train_scaled.columns, 2):
    # Create interaction term
    X_train_interactions['interaction'] = X_train_scaled[col1] * X_train_scaled[col2]
    X_val_interactions['interaction'] = X_val_scaled[col1] * X_val_scaled[col2]
    
    # Fit the LinearRegression model with the interaction term
    model = LinearRegression()
    model.fit(X_train_interactions, y_train)
    
    # Calculate the R-squared score on validation data
    r2_val = model.score(X_val_interactions, y_val)
    
    # Append results to the data structure
    interaction_scores.append({
        'pair': (col1, col2),
        'r2_val': r2_val
    })
    
    # Remove the interaction column for the next iteration
    X_train_interactions.drop(columns='interaction', inplace=True)
    X_val_interactions.drop(columns='interaction', inplace=True)

# Convert the results to a DataFrame and sort by R^2 score in descending order
interaction_scores_df = pd.DataFrame(interaction_scores)
top_7_interactions = interaction_scores_df.nlargest(7, 'r2_val')

# Print the top 7 interactions
print(top_7_interactions)

                          pair    r2_val
3          (LotArea, 1stFlrSF)  0.721111
2       (LotArea, TotalBsmtSF)  0.707165
5         (LotArea, GrLivArea)  0.669098
8        (LotArea, Fireplaces)  0.652970
36    (2ndFlrSF, TotRmsAbvGrd)  0.647299
17  (OverallCond, TotalBsmtSF)  0.642902
12     (OverallQual, 2ndFlrSF)  0.642232


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [6]:
# Your code here

# Loop over top 7 interactions

    # Extract column names from data structure

    # Construct new column name
    
    # Add new column to X_train and X_val

# Loop over the top 7 interactions
for _, row in top_7_interactions.iterrows():
    # Extract column names
    col1, col2 = row['pair']
    
    # Construct new column name
    interaction_name = f"{col1}_{col2}"
    
    # Add interaction columns to X_train and X_val
    X_train[interaction_name] = X_train_scaled[col1] * X_train_scaled[col2]
    X_val[interaction_name] = X_val_scaled[col1] * X_val_scaled[col2]

# Display the updated X_train to verify new columns
X_train.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
518,9531,6,5,794,882,914,1796,7,546,0,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
236,8773,7,5,1414,1414,0,1414,6,494,0,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
38,7922,5,7,1057,1057,0,1057,5,246,0,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
1034,6305,5,7,920,954,0,954,5,240,1,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
520,10800,4,7,0,694,600,1294,7,0,0,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [7]:
# Your code here

# Set up data structure

# Loop over all columns

    # Loop over degrees 2, 3, 4
        
        # Make a copy of X_train and X_val
    
        # Instantiate PolynomialFeatures with relevant degree
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
    
        # Find r-squared score on validation
    
        # Append to data structure

# Sort and subset the data structure to find the top 7

from sklearn.preprocessing import PolynomialFeatures

# Data structure to store results
polynomial_scores = []

# Loop over all columns in X_train
for col in X_train_scaled.columns:
    # Loop over degrees 2, 3, 4
    for degree in [2, 3, 4]:
        # Make a copy of X_train and X_val
        X_train_poly = X_train_scaled.copy()
        X_val_poly = X_val_scaled.copy()
        
        # Instantiate PolynomialFeatures with the relevant degree
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        
        # Fit and transform the specific column
        poly_train = poly.fit_transform(X_train_scaled[[col]])
        poly_val = poly.transform(X_val_scaled[[col]])
        
        # Convert the result to a DataFrame with proper column names
        col_names = [f"{col}^d" for d in range(2, degree + 1)]
        poly_train_df = pd.DataFrame(poly_train[:, 1:], columns=col_names, index=X_train.index)
        poly_val_df = pd.DataFrame(poly_val[:, 1:], columns=col_names, index=X_val.index)
        
        # Add the polynomial features to the DataFrame
        X_train_poly = pd.concat([X_train_poly.drop(columns=col), poly_train_df], axis=1)
        X_val_poly = pd.concat([X_val_poly.drop(columns=col), poly_val_df], axis=1)
        
        # Fit a LinearRegression model with the polynomial term
        model = LinearRegression()
        model.fit(X_train_poly, y_train)
        
        # Calculate the R-squared score on the validation data
        r2_val = model.score(X_val_poly, y_val)
        
        # Append results to the data structure
        polynomial_scores.append((col, degree, r2_val))

# Convert the results to a DataFrame and sort by R^2 in descending order
polynomial_scores_df = pd.DataFrame(polynomial_scores, columns=["col_name", "degree", "r2_val"])
top_7_polynomials = polynomial_scores_df.nlargest(7, 'r2_val')

# Print the top 7 polynomials
print(top_7_polynomials)


        col_name  degree    r2_val
20     GrLivArea       4  0.804633
19     GrLivArea       3  0.804154
16      2ndFlrSF       3  0.681615
17      2ndFlrSF       4  0.679184
15      2ndFlrSF       2  0.677017
12      1stFlrSF       2  0.653042
21  TotRmsAbvGrd       2  0.642568


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [8]:
# Your code here

# Filter out duplicates

# Loop over remaining results

    # Create polynomial terms
    
    # Concat new polynomials to X_train and X_val

# Filter out duplicates, keeping only the highest R^2 for each column
top_polynomials = top_7_polynomials.sort_values("r2_val", ascending=False).drop_duplicates(subset="col_name")

# Loop over the filtered results
for _, row in top_polynomials.iterrows():
    col = row['col_name']
    degree = int(row['degree'])
    
    # Instantiate PolynomialFeatures with the relevant degree
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    # Fit and transform the specific column
    poly_train = poly.fit_transform(X_train_scaled[[col]])
    poly_val = poly.transform(X_val_scaled[[col]])
    
    # Convert the result to a DataFrame with proper column names
    col_names = [f"{col}^{d}" for d in range(2, degree + 1)]
    poly_train_df = pd.DataFrame(poly_train[:, 1:], columns=col_names, index=X_train.index)
    poly_val_df = pd.DataFrame(poly_val[:, 1:], columns=col_names, index=X_val.index)
    
    # Add the new polynomial features to X_train and X_val
    X_train = pd.concat([X_train, poly_train_df], axis=1)
    X_val = pd.concat([X_val, poly_val_df], axis=1)

# Display the updated X_train to verify the new columns
X_train.head()
    

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,...,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF,GrLivArea^2,GrLivArea^3,GrLivArea^4,2ndFlrSF^2,2ndFlrSF^3,1stFlrSF^2,TotRmsAbvGrd^2
518,9531,6,5,794,882,914,1796,7,546,0,...,0.316257,0.325573,-0.125956,0.249115,0.124337,0.062058,1.591512,2.007775,0.647685,0.062845
236,8773,7,5,1414,1414,0,1414,6,494,0,...,0.295392,-0.426859,-0.510770,0.061132,-0.015115,0.003737,0.653077,-0.527772,0.411661,0.133609
38,7922,5,7,1057,1057,0,1057,5,246,0,...,0.793375,-0.016386,0.672141,0.892583,-0.843282,0.796704,0.653077,-0.527772,0.108241,0.963812
1034,6305,5,7,920,954,0,954,5,240,1,...,0.793375,-0.442323,0.672141,1.313340,-1.505101,1.724862,0.653077,-0.527772,0.370925,0.963812
520,10800,4,7,0,694,600,1294,7,0,0,...,0.138010,-3.302627,-0.860799,0.232043,-0.111777,0.053844,0.303075,0.166850,1.731650,0.062845


Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [9]:
# Your code here

X_train.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,...,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF,GrLivArea^2,GrLivArea^3,GrLivArea^4,2ndFlrSF^2,2ndFlrSF^3,1stFlrSF^2,TotRmsAbvGrd^2
518,9531,6,5,794,882,914,1796,7,546,0,...,0.316257,0.325573,-0.125956,0.249115,0.124337,0.062058,1.591512,2.007775,0.647685,0.062845
236,8773,7,5,1414,1414,0,1414,6,494,0,...,0.295392,-0.426859,-0.510770,0.061132,-0.015115,0.003737,0.653077,-0.527772,0.411661,0.133609
38,7922,5,7,1057,1057,0,1057,5,246,0,...,0.793375,-0.016386,0.672141,0.892583,-0.843282,0.796704,0.653077,-0.527772,0.108241,0.963812
1034,6305,5,7,920,954,0,954,5,240,1,...,0.793375,-0.442323,0.672141,1.313340,-1.505101,1.724862,0.653077,-0.527772,0.370925,0.963812
520,10800,4,7,0,694,600,1294,7,0,0,...,0.138010,-3.302627,-0.860799,0.232043,-0.111777,0.053844,0.303075,0.166850,1.731650,0.062845


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [10]:
# Your code here
# Your code here
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Create and train the model using the training data
final_model = LinearRegression()
final_model.fit(X_train, y_train)

# Predict on both training and validation data
y_train_pred = final_model.predict(X_train)
y_val_pred = final_model.predict(X_val)

# Calculate and print the R^2 scores
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Train R^2: {train_r2:.4f}")
print(f"Validation R^2: {val_r2:.4f}")

Train R^2: 0.8304
Validation R^2: 0.6944


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [11]:
# Your code here

from sklearn.feature_selection import RFE

# Test RFE with various numbers of features to select
n_features_list = [10, 20, 50, 100]  # Adjust based on dataset size
results = []

for n_features in n_features_list:
    # Initialize RFE with the LinearRegression model
    rfe = RFE(estimator=LinearRegression(), n_features_to_select=n_features)
    
    # Fit RFE on the training data
    rfe.fit(X_train, y_train)
    
    # Transform the datasets to keep only the selected features
    X_train_rfe = rfe.transform(X_train)
    X_val_rfe = rfe.transform(X_val)
    
    # Train a new LinearRegression model on the reduced dataset
    model = LinearRegression()
    model.fit(X_train_rfe, y_train)
    
    # Evaluate R^2 on train and validation data
    train_r2 = model.score(X_train_rfe, y_train)
    val_r2 = model.score(X_val_rfe, y_val)
    
    # Store results
    results.append((n_features, train_r2, val_r2))
    
    # Print results for this iteration
    print(f"n_features_to_select: {n_features}")
    print(f"  Train R^2: {train_r2:.4f}")
    print(f"  Validation R^2: {val_r2:.4f}")
    print("-" * 30)

# Optional: Convert results to a DataFrame for easier analysis
import pandas as pd
results_df = pd.DataFrame(results, columns=["n_features", "train_r2", "val_r2"])


n_features_to_select: 10
  Train R^2: 0.7330
  Validation R^2: 0.5464
------------------------------
n_features_to_select: 20
  Train R^2: 0.8156
  Validation R^2: 0.7035
------------------------------
n_features_to_select: 50
  Train R^2: 0.8304
  Validation R^2: 0.6944
------------------------------
n_features_to_select: 100
  Train R^2: 0.8304
  Validation R^2: 0.6944
------------------------------


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [12]:
# Your code here

# Your code here
from sklearn.linear_model import Lasso

# Test Lasso with various alpha values
alpha_values = [0.01, 0.1, 1, 10, 100]  # Adjust based on dataset
results = []

for alpha in alpha_values:
    # Initialize Lasso model with the given alpha
    lasso = Lasso(alpha=alpha, max_iter=10000, random_state=42)
    
    # Fit Lasso on the training data
    lasso.fit(X_train, y_train)
    
    # Evaluate R^2 on train and validation data
    train_r2 = lasso.score(X_train, y_train)
    val_r2 = lasso.score(X_val, y_val)
    
    # Store results
    results.append((alpha, train_r2, val_r2))
    
    # Print results for this iteration
    print(f"alpha: {alpha}")
    print(f"  Train R^2: {train_r2:.4f}")
    print(f"  Validation R^2: {val_r2:.4f}")
    print("-" * 30)

# Optional: Convert results to a DataFrame for easier analysis
import pandas as pd
results_df = pd.DataFrame(results, columns=["alpha", "train_r2", "val_r2"])


alpha: 0.01
  Train R^2: 0.8304
  Validation R^2: 0.6944
------------------------------
alpha: 0.1
  Train R^2: 0.8304
  Validation R^2: 0.6944
------------------------------
alpha: 1
  Train R^2: 0.8304
  Validation R^2: 0.6943
------------------------------
alpha: 10
  Train R^2: 0.8304
  Validation R^2: 0.6941
------------------------------
alpha: 100
  Train R^2: 0.8303
  Validation R^2: 0.6954
------------------------------


Compare the results. Which features would you choose to use?

# Your written answer here

### Recommendation
### Based on the results:

### RFE with 50 features gives the same performance as the full model, so you can use those features for simplicity.
Alternatively, Lasso with an appropriate alpha (e.g., 100) might result in slight regularization and potentially smaller coefficients, offering a marginal improvement in generalization.
For interpretability and practicality, RFE with 50 features is the best choice. It reduces complexity without compromising performance. If feature selection is not critical, sticking to the full linear regression model is equally valid.

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [ ]:
# Your code here


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [ ]:
# Your code here


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.